In [1]:
import numpy as np
import pandas as pd
import os
from collections import Counter
import subprocess
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import re

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV

from sklearn.decomposition import NMF
from sklearn.pipeline import make_pipeline

from sklearn import linear_model

In [2]:
twt_df = pd.read_csv('data/train.csv', encoding='latin1', index_col='ItemID')
twt_df.head()

,Sentiment,SentimentText
ItemID,,
1,0,is so sad for my APL frie...
2,0,I missed the New Moon trail...
3,1,omg its already 7:30 :O
4,0,.. Omgaga. Im sooo im gunna CRy. I'...
5,0,i think mi bf is cheating on me!!! ...


In [3]:
print(twt_df['Sentiment'].value_counts())
targets = twt_df['Sentiment']
twt_df.drop(columns=['Sentiment'], inplace=True)

1    56457
0    43532
Name: Sentiment, dtype: int64


In [4]:
train_twt, test_twt, train_targets, test_targets = train_test_split(twt_df, targets, test_size=0.2,
                                                                    stratify=targets)

In [5]:
def str_preprocessor(doc):
    return re.sub(r"(?u)@\w+", '', doc)

pattern = r"(?u)\b\w+[\-'.]?\w+\b|\b\d+[:]?\d+\b|\d"

stop_words = ['a', 'an', 'the']

In [36]:
vectorizer = TfidfVectorizer(analyzer='word', lowercase=True, stop_words = stop_words,
                             preprocessor=str_preprocessor, token_pattern=pattern)
X_train = vectorizer.fit_transform(train_twt['SentimentText'])

/Users/chanuwasaswamenakul/tensorflow/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:1089: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


In [11]:
print(vectorizer.get_stop_words())
print(len(vectorizer.vocabulary_))

frozenset({'a', 'an', 'the'})
65255


In [19]:
vocab = vectorizer.get_feature_names()

In [15]:
nmf_model = NMF(n_components=100)
nmf_model.fit(X_train)
X_train = nmf_model.transform(X_train)

In [20]:
nmf_model.components_.shape

(100, 65255)

In [21]:
component_df = pd.DataFrame(nmf_model.components_, columns=vocab)

In [35]:
component_df.iloc[33].nlargest(10)

just      5.842624
saw       0.252327
tweet     0.148705
say       0.129720
home      0.124726
made      0.122114
had       0.109913
wanted    0.106284
from      0.103854
woke      0.103146
Name: 33, dtype: float64